In [1]:
import os
import xarray as xr
import pandas as pd

In [2]:
#Palau EEZ
# Min. Lat 	1° 37' 17.1" N (1.6214°)  
# Min. Long 	129° 30' 31.7" E (129.5088°)  
	
# Max. Lat 	11° 33' 31.4" N (11.5587°)  
# Max. Long 	136° 57' 14.8" E (136.9541°)

#RMI EEZ
# Min. Lat 	1° 46' 38.3" N (1.7773°)  
# Min. Long 	157° 27' 37.9" E (157.4605°)  
	
# Max. Lat 	17° 56' 45.8" N (17.9461°)  
# Max. Long 	175° 31' 24.5" E (175.5235°)

buffer = 0.5
def filter_eez(df):
    filtered = df[(df["latitude"]>= min(1.6214-buffer,1.7773-buffer))&
                                (df["latitude"]<=max(11.5587+buffer,17.9461+buffer))&
                                (df["longitude"]<=max(175.5235+buffer,136.9541+buffer))&
                                (df["longitude"]>=min(129.5088-buffer,157.4605-buffer))]
    return filtered

In [3]:
def filter_palau(df):
    filtered = df[(df["latitude"]>= 1.25)&
                                (df["latitude"]<=13.00)&(df["longitude"]<=139.00)&
                                (df["longitude"]>=128)]
    return filtered

In [4]:
def processnc(ncDirectory):
    wave = xr.open_dataset(ncDirectory)
    wavefp, wavedp, wavehs = wave.fp.to_dataframe(), wave.dp.to_dataframe(), wave.hs.to_dataframe()
    wavefp, wavedp, wavehs = wavefp.dropna(), wavedp.dropna(), wavehs.dropna()
    wavefp = wavefp.reset_index()
    wavedp = wavedp.reset_index()
    wavehs = wavehs.reset_index()
    wavefp, wavedp, wavehs = filter_palau(wavefp), filter_palau(wavedp), filter_palau(wavehs)
    return wavefp,wavedp,wavehs

In [5]:
def processfp(ncDirectory):
    wave = xr.open_dataset(ncDirectory)
    wavefp = wave.fp.to_dataframe()
    wavefp = wavefp.dropna()
    wavefp = wavefp.reset_index()
    wavefp = filter_palau(wavefp)
    return wavefp

In [6]:
def processdp(ncDirectory):
    wave = xr.open_dataset(ncDirectory)
    wavedp = wave.dp.to_dataframe()
    wavedp = wavedp.dropna()
    wavedp = wavedp.reset_index()
    wavedp = filter_palau(wavedp)
    return wavedp

In [7]:
def processhs(ncDirectory):
    wave = xr.open_dataset(ncDirectory)
    wavehs = wave.hs.to_dataframe()
    wavehs = wavehs.dropna()
    wavehs = wavehs.reset_index()
    wavehs = filter_palau(wavehs)
    return wavehs

In [ ]:
import time

# l = os.listdir('/Volumes/PACIOOS/wave')
# l.sort()

master = None
#Change to directory the PACCSAP wave files are downloaded
for x in os.listdir('/Volumes/PACIOOS/wave'):
    st = time.time()
    print('/Volumes/PACIOOS/wave/' + x)
    
    wavefp, wavedp, wavehs = processnc('/Volumes/PACIOOS/wave/' + x)
    if master == None:
        master = [wavefp, wavedp, wavehs]
    else:
        master[0] = pd.concat([master[0],wavefp])
        master[1] = pd.concat([master[1],wavedp])
        master[2] = pd.concat([master[2],wavehs])
    et = time.time()
    elapsed_time = et - st
    print('Execution time:', elapsed_time, 'seconds')
    master[0].to_pickle("masterfp.pkl")
    master[1].to_pickle("masterdp.pkl")
    master[2].to_pickle("masterhs.pkl")

/Volumes/PACIOOS/wave/ww3.pac_4m.202205.nc
Execution time: 168.71138215065002 seconds
/Volumes/PACIOOS/wave/ww3.pac_4m.202210.nc
Execution time: 169.96666193008423 seconds
/Volumes/PACIOOS/wave/ww3.pac_4m.202301.nc
Execution time: 174.77187299728394 seconds
/Volumes/PACIOOS/wave/ww3.pac_4m.202207.nc
Execution time: 168.47843623161316 seconds
/Volumes/PACIOOS/wave/ww3.pac_4m.202211.nc
Execution time: 161.07746815681458 seconds
/Volumes/PACIOOS/wave/ww3.pac_4m.202212.nc
Execution time: 167.42707085609436 seconds
/Volumes/PACIOOS/wave/ww3.pac_4m.202209.nc
Execution time: 1674.6546070575714 seconds
/Volumes/PACIOOS/wave/ww3.pac_4m.202208.nc
Execution time: 180.9025809764862 seconds
/Volumes/PACIOOS/wave/ww3.pac_4m.202206.nc
Execution time: 168.2313838005066 seconds
/Volumes/PACIOOS/wave/ww3.pac_4m.197902.nc
Execution time: 162.47037386894226 seconds
/Volumes/PACIOOS/wave/ww3.pac_4m.197903.nc
Execution time: 181.42905116081238 seconds
/Volumes/PACIOOS/wave/ww3.pac_4m.197904.nc
Execution tim

In [40]:
from datetime import datetime
import time

masterfp = pd.read_pickle("masterfp.pkl")
datelist = masterfp["time"].dt.date.unique().tolist()
#Change to directory the PACCSAP wave files are downloaded
for x in os.listdir('/Volumes/PACIOOS/wave'):
    st = time.time()
    print('/Volumes/PACIOOS/wave/' + x)
    
    dateStr = x[11:-5]+x[-5:-3]+'01'
    date1 = datetime.strptime(dateStr, '%Y%m%d').date()
    dateStr = x[11:-5]+x[-5:-3]+'03'
    date2 = datetime.strptime(dateStr, '%Y%m%d').date()
    dateStr = x[11:-5]+x[-5:-3]+'06'
    date3 = datetime.strptime(dateStr, '%Y%m%d').date()
    if date1 in datelist or date2 in datelist or date3 in datelist:
        continue
    else:
        wavefp = processfp('/Volumes/PACIOOS/wave/' + x)
        masterfp = pd.concat([masterfp,wavefp])
        datelist.append(date1)
        datelist.append(date2)
        datelist.append(date3)
        masterfp.to_pickle("masterfp.pkl")
    et = time.time()
    elapsed_time = et - st
    print('Execution time:', elapsed_time, 'seconds')
    

/Volumes/PACIOOS/wave/ww3.pac_4m.202205.nc
/Volumes/PACIOOS/wave/ww3.pac_4m.202210.nc
/Volumes/PACIOOS/wave/ww3.pac_4m.202301.nc
/Volumes/PACIOOS/wave/ww3.pac_4m.202207.nc
/Volumes/PACIOOS/wave/ww3.pac_4m.202211.nc
/Volumes/PACIOOS/wave/ww3.pac_4m.202212.nc
/Volumes/PACIOOS/wave/ww3.pac_4m.202209.nc
/Volumes/PACIOOS/wave/ww3.pac_4m.202208.nc
/Volumes/PACIOOS/wave/ww3.pac_4m.202206.nc
/Volumes/PACIOOS/wave/ww3.pac_4m.197902.nc
/Volumes/PACIOOS/wave/ww3.pac_4m.197903.nc
/Volumes/PACIOOS/wave/ww3.pac_4m.197904.nc
/Volumes/PACIOOS/wave/ww3.pac_4m.197901.nc
/Volumes/PACIOOS/wave/ww3.pac_4m.197905.nc
/Volumes/PACIOOS/wave/ww3.pac_4m.197907.nc
/Volumes/PACIOOS/wave/ww3.pac_4m.197908.nc
/Volumes/PACIOOS/wave/ww3.pac_4m.197909.nc
/Volumes/PACIOOS/wave/ww3.pac_4m.197910.nc
/Volumes/PACIOOS/wave/ww3.pac_4m.198001.nc
/Volumes/PACIOOS/wave/ww3.pac_4m.198002.nc
/Volumes/PACIOOS/wave/ww3.pac_4m.198003.nc
/Volumes/PACIOOS/wave/ww3.pac_4m.198004.nc
/Volumes/PACIOOS/wave/ww3.pac_4m.198005.nc
/Volumes/PA

OSError: [Errno -101] NetCDF: HDF error: b'/Volumes/PACIOOS/wave/ww3.pac_4m.199405.nc'

In [12]:
from datetime import datetime
import time

masterdp = pd.read_pickle("masterdp.pkl")
datelist = masterdp["time"].dt.date.unique().tolist()
#Change to directory the PACCSAP wave files are downloaded
for x in os.listdir('/Volumes/PACIOOS/wave'):
    if x == 'ww3.pac_4m.199405.nc':
        continue
    try:
        st = time.time()
        print('/Volumes/PACIOOS/wave/' + x)

        dateStr = x[11:-5]+x[-5:-3]+'01'
        date1 = datetime.strptime(dateStr, '%Y%m%d').date()
        dateStr = x[11:-5]+x[-5:-3]+'03'
        date2 = datetime.strptime(dateStr, '%Y%m%d').date()
        dateStr = x[11:-5]+x[-5:-3]+'06'
        date3 = datetime.strptime(dateStr, '%Y%m%d').date()
        if date1 in datelist or date2 in datelist or date3 in datelist:
            continue
        else:
            wavedp = processdp('/Volumes/PACIOOS/wave/' + x)
            masterdp = pd.concat([masterdp,wavedp])
            datelist.append(date1)
            datelist.append(date2)
            datelist.append(date3)
            masterdp.to_pickle("masterdp.pkl")
            masterdp.to_csv('masterdp.csv',index=False)
        et = time.time()
        elapsed_time = et - st
        print('Execution time:', elapsed_time, 'seconds')
    except OSError as error:
        print(error)

/Volumes/PACIOOS/wave/ww3.pac_4m.202205.nc
/Volumes/PACIOOS/wave/ww3.pac_4m.202210.nc
/Volumes/PACIOOS/wave/ww3.pac_4m.202301.nc
/Volumes/PACIOOS/wave/ww3.pac_4m.202207.nc
/Volumes/PACIOOS/wave/ww3.pac_4m.202211.nc
/Volumes/PACIOOS/wave/ww3.pac_4m.202212.nc
/Volumes/PACIOOS/wave/ww3.pac_4m.202209.nc
/Volumes/PACIOOS/wave/ww3.pac_4m.202208.nc
/Volumes/PACIOOS/wave/ww3.pac_4m.202206.nc
/Volumes/PACIOOS/wave/ww3.pac_4m.197902.nc
/Volumes/PACIOOS/wave/ww3.pac_4m.197903.nc
/Volumes/PACIOOS/wave/ww3.pac_4m.197904.nc
/Volumes/PACIOOS/wave/ww3.pac_4m.197901.nc
/Volumes/PACIOOS/wave/ww3.pac_4m.197905.nc
/Volumes/PACIOOS/wave/ww3.pac_4m.197907.nc
/Volumes/PACIOOS/wave/ww3.pac_4m.197908.nc
/Volumes/PACIOOS/wave/ww3.pac_4m.197909.nc
/Volumes/PACIOOS/wave/ww3.pac_4m.197910.nc
/Volumes/PACIOOS/wave/ww3.pac_4m.198001.nc
/Volumes/PACIOOS/wave/ww3.pac_4m.198002.nc
/Volumes/PACIOOS/wave/ww3.pac_4m.198003.nc
/Volumes/PACIOOS/wave/ww3.pac_4m.198004.nc
/Volumes/PACIOOS/wave/ww3.pac_4m.198005.nc
/Volumes/PA

In [11]:
from datetime import datetime
import time

masterhs = pd.read_pickle("masterhs.pkl")
datelist = masterhs["time"].dt.date.unique().tolist()

#Change to directory the PACCSAP wave files are downloaded
for x in os.listdir('/Volumes/PACIOOS/wave'):
    if x == 'ww3.pac_4m.199405.nc':
        continue
    st = time.time()
    print('/Volumes/PACIOOS/wave/' + x)
    
    dateStr = x[11:-5]+x[-5:-3]+'01'
    date1 = datetime.strptime(dateStr, '%Y%m%d').date()
    dateStr = x[11:-5]+x[-5:-3]+'03'
    date2 = datetime.strptime(dateStr, '%Y%m%d').date()
    dateStr = x[11:-5]+x[-5:-3]+'06'
    date3 = datetime.strptime(dateStr, '%Y%m%d').date()
    if date1 in datelist or date2 in datelist or date3 in datelist:
        continue
    else:
        wavehs = processhs('/Volumes/PACIOOS/wave/' + x)
        masterhs = pd.concat([masterhs,wavehs])
        datelist.append(date1)
        datelist.append(date2)
        datelist.append(date3)
        masterhs.to_pickle("masterhs.pkl")
        masterhs.to_csv('masterhs.csv',index=False)
    et = time.time()
    elapsed_time = et - st
    print('Execution time:', elapsed_time, 'seconds')

/Volumes/PACIOOS/wave/ww3.pac_4m.202205.nc
/Volumes/PACIOOS/wave/ww3.pac_4m.202210.nc
/Volumes/PACIOOS/wave/ww3.pac_4m.202301.nc
/Volumes/PACIOOS/wave/ww3.pac_4m.202207.nc
/Volumes/PACIOOS/wave/ww3.pac_4m.202211.nc
/Volumes/PACIOOS/wave/ww3.pac_4m.202212.nc
/Volumes/PACIOOS/wave/ww3.pac_4m.202209.nc
/Volumes/PACIOOS/wave/ww3.pac_4m.202208.nc
/Volumes/PACIOOS/wave/ww3.pac_4m.202206.nc
/Volumes/PACIOOS/wave/ww3.pac_4m.197902.nc
/Volumes/PACIOOS/wave/ww3.pac_4m.197903.nc
/Volumes/PACIOOS/wave/ww3.pac_4m.197904.nc
/Volumes/PACIOOS/wave/ww3.pac_4m.197901.nc
/Volumes/PACIOOS/wave/ww3.pac_4m.197905.nc
/Volumes/PACIOOS/wave/ww3.pac_4m.197907.nc
/Volumes/PACIOOS/wave/ww3.pac_4m.197908.nc
/Volumes/PACIOOS/wave/ww3.pac_4m.197909.nc
/Volumes/PACIOOS/wave/ww3.pac_4m.197910.nc
/Volumes/PACIOOS/wave/ww3.pac_4m.198001.nc
/Volumes/PACIOOS/wave/ww3.pac_4m.198002.nc
/Volumes/PACIOOS/wave/ww3.pac_4m.198003.nc
/Volumes/PACIOOS/wave/ww3.pac_4m.198004.nc
/Volumes/PACIOOS/wave/ww3.pac_4m.198005.nc
/Volumes/PA

Execution time: 115.83771991729736 seconds
/Volumes/PACIOOS/wave/ww3.pac_4m.199110.nc
Execution time: 117.017902135849 seconds
/Volumes/PACIOOS/wave/ww3.pac_4m.199201.nc
Execution time: 126.66835904121399 seconds
/Volumes/PACIOOS/wave/ww3.pac_4m.199202.nc
Execution time: 122.39673614501953 seconds
/Volumes/PACIOOS/wave/ww3.pac_4m.199203.nc
Execution time: 127.10292887687683 seconds
/Volumes/PACIOOS/wave/ww3.pac_4m.199204.nc
Execution time: 122.13036394119263 seconds
/Volumes/PACIOOS/wave/ww3.pac_4m.199205.nc
Execution time: 124.86441397666931 seconds
/Volumes/PACIOOS/wave/ww3.pac_4m.199207.nc
Execution time: 130.43239879608154 seconds
/Volumes/PACIOOS/wave/ww3.pac_4m.199208.nc
Execution time: 134.6205976009369 seconds
/Volumes/PACIOOS/wave/ww3.pac_4m.199209.nc
Execution time: 132.54597997665405 seconds
/Volumes/PACIOOS/wave/ww3.pac_4m.199210.nc
Execution time: 156.59450912475586 seconds
/Volumes/PACIOOS/wave/ww3.pac_4m.199301.nc
Execution time: 138.28143167495728 seconds
/Volumes/PACIO

In [ ]:
import time

l = os.listdir('/Volumes/PACIOOS/wave')
l.sort()

master = None
#Change to directory the PACCSAP wave files are downloaded
for x in os.listdir('/Volumes/PACIOOS/wave'):
    st = time.time()
    print('/Volumes/PACIOOS/wave/' + x)
    
    wavefp = processnc('/Volumes/PACIOOS/wave/' + x)
    if master == None:
        master = wavefp
    else:
        master = pd.concat([master,wavefp])

    et = time.time()
    elapsed_time = et - st
    print('Execution time:', elapsed_time, 'seconds')
    master.to_pickle("masterfp.pkl")
    masterfp.to_csv('masterfp.csv',index=False)


In [44]:
#Master is the master dataframe (like masterfp), monthInterval is # month to space out
#name is name of file to save as
def monthAverage(master,monthInterval,name,coordinateList):
    masterInterval = None
    for count, coord in enumerate(coordinateList):
        print(count,coord)
        coordinateSlice = master[(master['latitude'] == coord[0])&(master['longitude'] ==coord[1])]
        if masterInterval is None:
            intermediate = coordinateSlice.groupby(pd.Grouper(key="time", freq=str(monthInterval)+"M", origin='1979-01-01')).mean().reset_index()
            masterInterval = intermediate.set_index(['latitude','longitude'])            
        else:
            intermediate = coordinateSlice.groupby(pd.Grouper(key="time", freq=str(monthInterval)+"M", origin='1979-01-01')).mean().reset_index()
            masterInterval = pd.concat([masterInterval,intermediate.set_index(['latitude','longitude'])])


    masterInterval.to_pickle(name)
    return masterInterval

In [ ]:
#legacy code to do it manually
# masterThreeMonthfp = None
# for count, coord in enumerate(coordinateList):
#     print(count,coord)
#     coordinateSlice = masterfp[(masterfp['latitude'] == coord[0])&(masterfp['longitude'] ==coord[1])]
#     if masterThreeMonthfp is None:

#         intermediate = coordinateSlice.groupby(pd.Grouper(key="time", freq="3M", origin='1979-01-01')).mean().reset_index()
#         masterThreeMonthfp = intermediate.set_index(['latitude','longitude'])
#     else:
#         threeMonthfp = coordinateSlice.groupby(pd.Grouper(key="time", freq="3M", origin='1979-01-01')).mean().set_index(['latitude','longitude'])
        
#         masterThreeMonthfp = pd.concat([masterThreeMonthfp,threeMonthfp])

In [52]:
import geopandas as gpd
from shapely.geometry import Point
df_eez = gpd.read_file('palauEEZ.geojson')
eez = df_eez['geometry'].values[0]

def palau_eez(df):
    in_palau = []
    longlat = df[['longitude','latitude']].values.tolist()
    for x in longlat:
        point = Point(x[0],x[1])
        if eez.contains(point) or point.within(eez):
            in_palau.append((x[0],x[1]))
    return df[df[['longitude', 'latitude']].apply(tuple, axis=1).isin(in_palau)]

In [23]:
masterdp = pd.read_pickle("masterdp.pkl")
masterdp = palau_eez(masterdp)
coordinates = masterdp[['latitude','longitude']]
coordinates = coordinates.drop_duplicates()
coordinates['cord'] = list(zip(coordinates.latitude, coordinates.longitude))
coordinateList = coordinates.cord.tolist()

monthAverage(masterdp,3,"masterThreeMonthdp.pkl",coordinateList)

In [46]:
masterhs = pd.read_pickle("masterhs.pkl")
masterhs = palau_eez(masterhs)
coordinates = masterhs[['latitude','longitude']]
coordinates = coordinates.drop_duplicates()
coordinates['cord'] = list(zip(coordinates.latitude, coordinates.longitude))
coordinateList = coordinates.cord.tolist()

monthAverage(masterhs,3,"masterThreeMonthhs.pkl",coordinateList)

masterfp = pd.read_pickle("masterfp.pkl")
masterfp = palau_eez(masterfp)
coordinates = masterfp[['latitude','longitude']]
coordinates = coordinates.drop_duplicates()
coordinates['cord'] = list(zip(coordinates.latitude, coordinates.longitude))
coordinateList = coordinates.cord.tolist()

monthAverage(masterfp,3,"masterThreeMonthfp.pkl",coordinateList)

0 (6.333515167236328, 133.800048828125)
1 (6.333515167236328, 133.86671447753906)
2 (6.333515167236328, 133.93338012695312)
3 (6.333515167236328, 134.0000457763672)
4 (6.333515167236328, 134.06671142578125)
5 (6.333515167236328, 134.1333770751953)
6 (6.400181770324707, 133.73338317871094)
7 (6.400181770324707, 133.800048828125)
8 (6.400181770324707, 133.86671447753906)
9 (6.400181770324707, 133.93338012695312)
10 (6.400181770324707, 134.0000457763672)
11 (6.400181770324707, 134.06671142578125)
12 (6.400181770324707, 134.1333770751953)
13 (6.400181770324707, 134.20004272460938)
14 (6.400181770324707, 134.26670837402344)
15 (6.400181770324707, 134.3333740234375)
16 (6.400181770324707, 134.40003967285156)
17 (6.466848850250244, 133.66671752929688)
18 (6.466848850250244, 133.73338317871094)
19 (6.466848850250244, 133.800048828125)
20 (6.466848850250244, 133.86671447753906)
21 (6.466848850250244, 133.93338012695312)
22 (6.466848850250244, 134.0000457763672)
23 (6.466848850250244, 134.066711

190 (6.933517932891846, 134.40003967285156)
191 (6.933517932891846, 134.4667205810547)
192 (6.933517932891846, 134.53338623046875)
193 (6.933517932891846, 134.6000518798828)
194 (6.933517932891846, 134.66671752929688)
195 (6.933517932891846, 134.73338317871094)
196 (6.933517932891846, 134.800048828125)
197 (6.933517932891846, 134.86671447753906)
198 (6.933517932891846, 134.93338012695312)
199 (6.933517932891846, 135.0000457763672)
200 (6.933517932891846, 135.06671142578125)
201 (6.933517932891846, 135.1333770751953)
202 (6.933517932891846, 135.20005798339844)
203 (6.933517932891846, 135.2667236328125)
204 (6.933517932891846, 135.33338928222656)
205 (6.933517932891846, 135.40005493164062)
206 (6.933517932891846, 135.4667205810547)
207 (7.000185012817383, 133.40003967285156)
208 (7.000185012817383, 133.46670532226562)
209 (7.000185012817383, 133.5333709716797)
210 (7.000185012817383, 133.60003662109375)
211 (7.000185012817383, 133.66671752929688)
212 (7.000185012817383, 133.7333831787109

379 (7.266852855682373, 135.33338928222656)
380 (7.266852855682373, 135.40005493164062)
381 (7.266852855682373, 135.4667205810547)
382 (7.266852855682373, 135.53338623046875)
383 (7.266852855682373, 135.6000518798828)
384 (7.266852855682373, 135.66671752929688)
385 (7.266852855682373, 135.73338317871094)
386 (7.266852855682373, 135.800048828125)
387 (7.266852855682373, 135.86671447753906)
388 (7.266852855682373, 135.9333953857422)
389 (7.266852855682373, 136.00006103515625)
390 (7.266852855682373, 136.0667266845703)
391 (7.33351993560791, 133.46670532226562)
392 (7.33351993560791, 133.5333709716797)
393 (7.33351993560791, 133.60003662109375)
394 (7.33351993560791, 133.66671752929688)
395 (7.33351993560791, 133.73338317871094)
396 (7.33351993560791, 133.800048828125)
397 (7.33351993560791, 133.86671447753906)
398 (7.33351993560791, 133.93338012695312)
399 (7.33351993560791, 134.0000457763672)
400 (7.33351993560791, 134.06671142578125)
401 (7.33351993560791, 134.1333770751953)
402 (7.333

568 (7.6001877784729, 134.0000457763672)
569 (7.6001877784729, 134.06671142578125)
570 (7.6001877784729, 134.1333770751953)
571 (7.6001877784729, 134.20004272460938)
572 (7.6001877784729, 134.26670837402344)
573 (7.6001877784729, 134.3333740234375)
574 (7.6001877784729, 134.40003967285156)
575 (7.6001877784729, 134.4667205810547)
576 (7.6001877784729, 134.53338623046875)
577 (7.6001877784729, 134.66671752929688)
578 (7.6001877784729, 134.73338317871094)
579 (7.6001877784729, 134.800048828125)
580 (7.6001877784729, 134.86671447753906)
581 (7.6001877784729, 134.93338012695312)
582 (7.6001877784729, 135.0000457763672)
583 (7.6001877784729, 135.06671142578125)
584 (7.6001877784729, 135.1333770751953)
585 (7.6001877784729, 135.20005798339844)
586 (7.6001877784729, 135.2667236328125)
587 (7.6001877784729, 135.33338928222656)
588 (7.6001877784729, 135.40005493164062)
589 (7.6001877784729, 135.4667205810547)
590 (7.6001877784729, 135.53338623046875)
591 (7.6001877784729, 135.6000518798828)
592

758 (7.866856098175049, 134.0000457763672)
759 (7.866856098175049, 134.06671142578125)
760 (7.866856098175049, 134.1333770751953)
761 (7.866856098175049, 134.20004272460938)
762 (7.866856098175049, 134.26670837402344)
763 (7.866856098175049, 134.3333740234375)
764 (7.866856098175049, 134.40003967285156)
765 (7.866856098175049, 134.4667205810547)
766 (7.866856098175049, 134.53338623046875)
767 (7.866856098175049, 134.6000518798828)
768 (7.866856098175049, 134.66671752929688)
769 (7.866856098175049, 134.73338317871094)
770 (7.866856098175049, 134.800048828125)
771 (7.866856098175049, 134.86671447753906)
772 (7.866856098175049, 134.93338012695312)
773 (7.866856098175049, 135.0000457763672)
774 (7.866856098175049, 135.06671142578125)
775 (7.866856098175049, 135.1333770751953)
776 (7.866856098175049, 135.20005798339844)
777 (7.866856098175049, 135.2667236328125)
778 (7.866856098175049, 135.33338928222656)
779 (7.866856098175049, 135.40005493164062)
780 (7.866856098175049, 135.4667205810547)

...

1716 (9.066862106323242, 135.86671447753906)
1717 (9.066862106323242, 135.9333953857422)
1718 (9.066862106323242, 136.00006103515625)
1719 (9.066862106323242, 136.0667266845703)
1720 (9.066862106323242, 136.13339233398438)
1721 (9.066862106323242, 136.20005798339844)
1722 (9.066862106323242, 136.2667236328125)
1723 (9.066862106323242, 136.33338928222656)
1724 (9.066862106323242, 136.40005493164062)
1725 (9.066862106323242, 136.4667205810547)
1726 (9.066862106323242, 136.53338623046875)
1727 (9.066862106323242, 136.6000518798828)
1728 (9.066862106323242, 136.66671752929688)
1729 (9.066862106323242, 136.7333984375)
1730 (9.066862106323242, 136.80006408691406)
1731 (9.066862106323242, 136.86672973632812)
1732 (9.066862106323242, 136.9333953857422)
1733 (9.066862106323242, 137.00006103515625)
1734 (9.066862106323242, 137.0667266845703)
1735 (9.066862106323242, 137.13339233398438)
1736 (9.066862106323242, 137.20005798339844)
1737 (9.066862106323242, 137.2667236328125)
1738 (9.06686210632324

1901 (9.266862869262695, 138.9333953857422)
1902 (9.33353042602539, 136.13339233398438)
1903 (9.33353042602539, 136.20005798339844)
1904 (9.33353042602539, 136.2667236328125)
1905 (9.33353042602539, 136.33338928222656)
1906 (9.33353042602539, 136.40005493164062)
1907 (9.33353042602539, 136.4667205810547)
1908 (9.33353042602539, 136.53338623046875)
1909 (9.33353042602539, 136.6000518798828)
1910 (9.33353042602539, 136.66671752929688)
1911 (9.33353042602539, 136.7333984375)
1912 (9.33353042602539, 136.80006408691406)
1913 (9.33353042602539, 136.86672973632812)
1914 (9.33353042602539, 136.9333953857422)
1915 (9.33353042602539, 137.00006103515625)
1916 (9.33353042602539, 137.0667266845703)
1917 (9.33353042602539, 137.13339233398438)
1918 (9.33353042602539, 137.20005798339844)
1919 (9.33353042602539, 137.2667236328125)
1920 (9.33353042602539, 137.33338928222656)
1921 (9.33353042602539, 137.40005493164062)
1922 (9.33353042602539, 137.46673583984375)
1923 (9.33353042602539, 137.5334014892578)

2088 (9.600197792053223, 138.2000732421875)
2089 (9.600197792053223, 138.26673889160156)
2090 (9.600197792053223, 138.33340454101562)
2091 (9.600197792053223, 138.4000701904297)
2092 (9.600197792053223, 138.46673583984375)
2093 (9.600197792053223, 138.5334014892578)
2094 (9.600197792053223, 138.60006713867188)
2095 (9.600197792053223, 138.66673278808594)
2096 (9.600197792053223, 138.7333984375)
2097 (9.600197792053223, 138.80006408691406)
2098 (9.600197792053223, 138.86672973632812)
2099 (9.600197792053223, 138.9333953857422)
2100 (9.666865348815918, 136.66671752929688)
2101 (9.666865348815918, 136.7333984375)
2102 (9.666865348815918, 136.80006408691406)
2103 (9.666865348815918, 136.86672973632812)
2104 (9.666865348815918, 136.9333953857422)
2105 (9.666865348815918, 137.00006103515625)
2106 (9.666865348815918, 137.0667266845703)
2107 (9.666865348815918, 137.13339233398438)
2108 (9.666865348815918, 137.20005798339844)
2109 (9.666865348815918, 137.2667236328125)
2110 (9.666865348815918, 

2273 (10.000200271606445, 137.7333984375)
2274 (10.000200271606445, 137.80006408691406)
2275 (10.000200271606445, 137.86672973632812)
2276 (10.000200271606445, 137.9333953857422)
2277 (10.000200271606445, 138.00006103515625)
2278 (10.000200271606445, 138.0667266845703)
2279 (10.000200271606445, 138.13339233398438)
2280 (10.000200271606445, 138.2000732421875)
2281 (10.000200271606445, 138.26673889160156)
2282 (10.000200271606445, 138.33340454101562)
2283 (10.000200271606445, 138.4000701904297)
2284 (10.000200271606445, 138.46673583984375)
2285 (10.000200271606445, 138.5334014892578)
2286 (10.000200271606445, 138.60006713867188)
2287 (10.000200271606445, 138.66673278808594)
2288 (10.000200271606445, 138.7333984375)
2289 (10.000200271606445, 138.80006408691406)
2290 (10.000200271606445, 138.86672973632812)
2291 (10.000200271606445, 138.9333953857422)
2292 (10.066866874694824, 137.13339233398438)
2293 (10.066866874694824, 137.20005798339844)
2294 (10.066866874694824, 137.2667236328125)
229

time        fp
latitude  longitude                      
6.333515  133.800049 1979-01-31  0.112321
          133.800049 1979-04-30  0.106209
          133.800049 1979-07-31  0.116169
          133.800049 1979-10-31  0.111874
          133.800049 1980-01-31  0.110579
...                         ...       ...
NaN       NaN        2022-01-31       NaN
          NaN        2022-04-30       NaN
10.466869 138.933395 2022-07-31  0.119366
          138.933395 2022-10-31  0.111005
          138.933395 2023-01-31  0.101705

[425685 rows x 2 columns]

In [75]:
#Remainder code used to create csv files from pkl files
from datetime import datetime, date
#2022-05-01 00:00:00.000000000
hs['date'] = hs['time'].astype(str) + " 00:00:00.000000000"

In [77]:
hs = pd.read_pickle('masterThreeMonthhs.pkl')
hs = hs.reset_index()
#this line may or may not be needed depending on how masterThreeMonthhs.pkl was filtered previously
# hs = palau_eez(hs)
hs = hs[['time','latitude','longitude','hs']]
hs['time'] = hs['time'].astype(str) + " 00:00:00.000000000"
hs.to_csv('masterhs.csv',index=False)
print(hs)
del hs

dp = pd.read_pickle('masterThreeMonthdp.pkl')
dp = dp.reset_index()
# dp = palau_eez(dp)
dp = dp[['time','latitude','longitude','dp']]
dp['time'] = dp['time'].astype(str) + " 00:00:00.000000000"
dp.to_csv('masterdp.csv',index=False)
print(dp)
del dp

fp = pd.read_pickle('masterThreeMonthfp.pkl')
fp = fp.reset_index()
# fp = palau_eez(fp)
fp = fp[['time','latitude','longitude','fp']]
fp['time'] = fp['time'].astype(str) + " 00:00:00.000000000"
fp.to_csv('masterhs.csv',index=False)
print(fp)
del fp

/opt/anaconda3/envs/cmip6/lib/python3.9/site-packages/shapely/predicates.py:540: RuntimeWarning: invalid value encountered in contains
  return lib.contains(a, b, **kwargs)


                                 time  latitude   longitude        hs
0       1979-01-31 00:00:00.000000000  6.333515  133.800049  1.734188
1       1979-04-30 00:00:00.000000000  6.333515  133.800049  1.525744
2       1979-07-31 00:00:00.000000000  6.333515  133.800049  0.996297
3       1979-10-31 00:00:00.000000000  6.333515  133.800049  1.245973
4       1980-01-31 00:00:00.000000000  6.333515  133.800049  1.812151
...                               ...       ...         ...       ...
312288  1994-01-31 00:00:00.000000000  9.133529  135.866714  1.935487
312289  1994-04-30 00:00:00.000000000  9.133529  135.866714  1.771041
312402  2022-07-31 00:00:00.000000000  9.133529  135.866714  1.070410
312403  2022-10-31 00:00:00.000000000  9.133529  135.866714  1.088823
312404  2023-01-31 00:00:00.000000000  9.133529  135.866714  1.918151

[72930 rows x 4 columns]


/opt/anaconda3/envs/cmip6/lib/python3.9/site-packages/shapely/predicates.py:540: RuntimeWarning: invalid value encountered in contains
  return lib.contains(a, b, **kwargs)


                                 time  latitude   longitude          dp
0       1979-01-31 00:00:00.000000000  6.333515  133.800049   51.101078
1       1979-04-30 00:00:00.000000000  6.333515  133.800049   67.703186
2       1979-07-31 00:00:00.000000000  6.333515  133.800049   93.758064
3       1979-10-31 00:00:00.000000000  6.333515  133.800049  141.166672
4       1980-01-31 00:00:00.000000000  6.333515  133.800049   51.940861
...                               ...       ...         ...         ...
312288  1994-01-31 00:00:00.000000000  9.133529  135.866714   44.467743
312289  1994-04-30 00:00:00.000000000  9.133529  135.866714   53.062733
312402  2022-07-31 00:00:00.000000000  9.133529  135.866714   73.360252
312403  2022-10-31 00:00:00.000000000  9.133529  135.866714  139.422913
312404  2023-01-31 00:00:00.000000000  9.133529  135.866714   74.337860

[72930 rows x 4 columns]


/opt/anaconda3/envs/cmip6/lib/python3.9/site-packages/shapely/predicates.py:540: RuntimeWarning: invalid value encountered in contains
  return lib.contains(a, b, **kwargs)


                                 time  latitude   longitude        fp
0       1979-01-31 00:00:00.000000000  6.333515  133.800049  0.112321
1       1979-04-30 00:00:00.000000000  6.333515  133.800049  0.106209
2       1979-07-31 00:00:00.000000000  6.333515  133.800049  0.116169
3       1979-10-31 00:00:00.000000000  6.333515  133.800049  0.111874
4       1980-01-31 00:00:00.000000000  6.333515  133.800049  0.110579
...                               ...       ...         ...       ...
312288  1994-01-31 00:00:00.000000000  9.133529  135.866714  0.107726
312289  1994-04-30 00:00:00.000000000  9.133529  135.866714  0.107080
312402  2022-07-31 00:00:00.000000000  9.133529  135.866714  0.117318
312403  2022-10-31 00:00:00.000000000  9.133529  135.866714  0.109336
312404  2023-01-31 00:00:00.000000000  9.133529  135.866714  0.100510

[72930 rows x 4 columns]


In [33]:
masterhs = pd.read_pickle("masterhs.pkl")
coordinates = masterhs[['latitude','longitude']]
coordinates = coordinates.drop_duplicates()
coordinates['cord'] = list(zip(coordinates.latitude, coordinates.longitude))
coordinateList = coordinates.cord.tolist()